### Notebook Pour une rentrée en sciences 2026

In [6]:
import openmeteo_requests

import requests
import pandas as pd
import requests_cache
import os
from retry_requests import retry
from datetime import date

In [88]:
def get_open_meteo_url(longitude, latitude, year_start, year_end, daily_variables):
    """
    Récupération de l'url de l'API Open-Météo

    Parameters
    ----------
    longitude : float
        DESCRIPTION.
    latitude : float
        DESCRIPTION.
    year : int
        DESCRIPTION.
    hourly_variables : list of str or str
        DESCRIPTION.

    Returns
    -------
    url : str
        DESCRIPTION.

    """
    if isinstance(daily_variables, list):
        daily_variables = ','.join(daily_variables)
    tod = pd.Timestamp(date.today())
    
    # Si l'année demandée n'est pas terminée, il faut modifier les périodes requêtées
    end_month, end_day = 12, 31
    if year_end == tod.year:
        end_day = tod.strftime('%d')
        end_month = tod.strftime('%m')
        
    url = 'https://archive-api.open-meteo.com/v1/archive?latitude={}&longitude={}&start_date={}-01-01&end_date={}-{}-{}&daily={}&timezone=Europe%2FBerlin'.format(latitude,longitude,year_start,year_end,end_month,end_day,daily_variables)
    # print(url)
    return url


def open_meteo_historical_data(longitude, latitude, year_start, year_end, daily_variables=["snowfall_sum", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean"], force=False, save = False):
    """
    Ouverture des fichiers meteo

    Parameters
    ----------
    longitude : float
        DESCRIPTION.
    latitude : float
        DESCRIPTION.
    year : int
        DESCRIPTION.
    hourly_variables : str or list of str, optional
        DESCRIPTION. The default is ['temperature_2m','direct_radiation_instant'].
    force : boolean, optional
        DESCRIPTION. The default is False.

    Returns
    -------
    data : pandas DataFrame
        DESCRIPTION.

    """
    # TODO : peut-etre mettre un nom de ville en entrée et en faire des nom de sauvegarde plus lisible
    if isinstance(daily_variables, list):
        daily_variables_str = ','.join(daily_variables)
    else:
        daily_variables_str = daily_variables
        
    save_path = os.path.join('data','Open-Meteo')
    save_name = '{}_{}_{}_{}.csv'.format(daily_variables_str, year_start, year_end, longitude, latitude)
    save_name_units = '{}_{}_{}_{}_units.txt'.format(daily_variables_str, year_start, year_end, longitude, latitude)

    if save_name not in os.listdir(save_path) or force:
        url = get_open_meteo_url(longitude, latitude, year_start, year_end, daily_variables)
        response = requests.get(url)
        print("Requesting data from Open-Meteo API...")
        json_data = response.json()
        if response.status_code==429:
            print("Too many requests. Waiting before retrying...")
            return None
        data = pd.DataFrame().from_dict(json_data.get('daily'))
        data.to_csv(os.path.join(save_path,save_name), index=False)
        
    data = pd.read_csv(os.path.join(save_path,save_name))
    data = data.set_index('time')
    data.index = pd.to_datetime(data.index)
    return data

In [40]:
pref = pd.read_csv("data/pref_lat_lon.csv", sep=";")
pref["lat"] = pref["Geo Point"].apply(lambda x: float(x.split(",")[0]))
pref["lon"] = pref["Geo Point"].apply(lambda x: float(x.split(",")[1]))
pref = pref[['Code INSEE', 'Commune', 'Service', 'lat', 'lon']]
# pref = pref[pref.Service == 'Préfecture']

seuils_df = pd.read_csv("data/seuils_canicules.csv", sep=",")

In [53]:
def make_plot(df, col, pref_name, out_path=None):
    import matplotlib.pyplot as plt

    dict_col = {
        'temperature_2m_mean': 'Tempature moyenne 2m (°C)',
        'snowfall_sum': 'Chute de neige annuelle (cm)',
        'canicule': 'Nombre de jours de canicule',
    }

    plt.figure(figsize=(12, 6))
    plt.plot(df['year'], df[col], label=col)
    plt.title(f"{dict_col.get(col, col)} : {pref_name}")
    plt.xlabel("Année")
    plt.ylabel(dict_col.get(col, col))
    plt.legend()
    plt.grid()
    if out_path is not None:
        plt.savefig(out_path)
    plt.close()

In [ ]:
lons = pref[pref.Service != 'Sous-préfecture'].lon.to_list()
lats = pref[pref.Service != 'Sous-préfecture'].lat.to_list()
communes = pref[pref.Service != 'Sous-préfecture'].Commune.to_list()
pref['Code INSEE'] = pref['Code INSEE'].astype(str).str.zfill(5)
code_insee = pref[pref.Service != 'Sous-préfecture']['Code INSEE'].to_list()
year_start, year_end = 1950, 2025

In [92]:
df_final = []
for i in range(len(lons)):
    lon, lat, name, code = lons[i], lats[i], communes[i], code_insee[i]
    print(f"Processing {name} ({code})")
    dept = code[:2]
    data = open_meteo_historical_data(lon, lat, year_start, year_end, 
                                      daily_variables=["snowfall_sum", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean"], force=False, save=False)
    if data is None:
        print(f"Skipping {name} ({code}) due to request issues.")
        break
    data['Commune'] = name
    data['Departement'] = dept
    data = data.reset_index()
    data = data.merge(seuils_df, left_on='Departement', right_on='dep', how='left')
    data['canicule'] = (data['temperature_2m_max'] >= data['smax'])&(data['temperature_2m_min'] >= data['smin'])
    data = data.set_index('time')
    data = data.resample('YS').agg({'canicule':'sum', 
                                                'temperature_2m_mean':'mean',
                                                'snowfall_sum':'sum', 
                                                'Commune':'first',
                                                'Departement':'first'})
    data['year'] = data.index.year
    os.makedirs(f"output/Dept_{dept}", exist_ok=True)
    make_plot(data, 'temperature_2m_mean', name, out_path=f"output/Dept_{dept}/{code}_temperature_2m_mean.png")
    make_plot(data, 'snowfall_sum', name, out_path=f"output/Dept_{dept}/{code}_snowfall_sum.png")
    make_plot(data, 'canicule', name, out_path=f"output/Dept_{dept}/{code}_canicule.png")
    df_final.append(data.copy())
df_final = pd.concat(df_final)
os.makedirs(os.path.join('data','Open-Meteo'), exist_ok=True)
df_final.to_csv(os.path.join('data','Open-Meteo',f'all_prefectures_{year_start}_{year_end}_yearly.csv'))

Processing Orléans (45234)
Processing Blois (41018)
Processing Carcassonne (11069)
Processing Saint-Lô (50502)
Processing Foix (09122)
Processing Tarbes (65440)
Requesting data from Open-Meteo API...
Processing Nevers (58194)
Requesting data from Open-Meteo API...
Too many requests. Waiting before retrying...
Skipping Nevers (58194) due to request issues.


In [91]:
df_final.groupby('Commune').agg({'canicule':'mean', 
                                'temperature_2m_mean':'mean',
                                'snowfall_sum':'mean'})

,canicule,temperature_2m_mean,snowfall_sum
Commune,,,
Blois,0.500000,11.546975,9.761316
Carcassonne,0.697368,13.471551,24.487105
Foix,0.000000,11.678061,110.524474
Orléans,0.618421,11.293719,12.013289
Saint-Lô,0.552632,11.274107,12.021579
